<a href="https://colab.research.google.com/github/mad-scientist-in-training/BIOF509-Final-Project/blob/Project-Code/BIOF509_Final_Project_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# 1. import all necessary packages
import numpy as np
import pandas as pd
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [ ]:
# 2. import dataset
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.environ['KAGGLEC_CONFIG_DIR'] = "content/gdrive/My Drive/kaggle_dataset"
%cd /content/gdrive/My Drive/kaggle_dataset
#copy the API command from kaggle and unzip the file
!kaggle datasets download -d ruslankl/early-biomarkers-of-parkinsons-disease
!unzip early-biomarkers-of-parkinsons-disease
PD_df = pd.read_csv(r'dataset.csv')
#optional - visualize dataset to understand various parameters before manipulation
print(PD_df)
len(PD_df)
PD_df.head()

In [12]:
# 3. manipulate the dataset
PD_df_1 = PD_df.drop(PD_df.columns[1:41], axis=1)
PD_DF = PD_df_1.rename({' Participant  code ':'Participant_Type'}, axis=1)
PD_DF['class'] = PD_DF.Participant_Type.str[0]
target = PD_DF.pop('class')
PD_DF.insert(0, 'class', target)
PD_DF_new = PD_DF.drop(PD_DF.columns[1], axis=1)
PD_DF_new["class"] = PD_DF_new["class"].replace('P', 0)
PD_DF_new["class"] = PD_DF_new["class"].replace('H', 1)
PD_DF_new["class"] = PD_DF_new["class"].replace('R', 2)
col_names = list(PD_DF_new)
PD_DF_new['class']
col_names = list(PD_DF_new)
df_scaled = PD_DF_new.copy()
all_cols_no_class = df_scaled[col_names]
all_cols_no_class = PD_DF_new.loc[:, PD_DF_new.columns !='class']
scaler = StandardScaler().fit(all_cols_no_class.values)
scaled_all_cols_no_class = scaler.transform(all_cols_no_class.values)
df_scaled = pd.DataFrame(scaled_all_cols_no_class)
df_scaled = pd.concat([df_scaled, PD_DF_new['class']], axis=1)
df_scaled_new = df_scaled.drop(df_scaled.loc[df_scaled['class']==2].index)
df_new_1 = df_scaled_new.rename(columns={'class':'24'})

In [15]:
# 4. create, train, test random forest classifier
X = df_new_1.iloc[:, 0:23].values
y = df_new_1.iloc[:, 24].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)
classifier = RandomForestClassifier(max_depth=32, n_estimators=100)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

In [ ]:
# 5. measure the accuracy of the algorithm
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(accuracy_score(y_pred, y_test))

In [ ]:
# 5. optional: optimize parameters using GridSearchCV
parameters = {"n_estimators":[5,10,50,100,250],
              "max_depth":[2,4,8,16,32,None]}
cv = GridSearchCV(classifier, parameters, cv=5)
cv.fit(X_train, y_train)
def display(results):
  print(f'Best parameters are: {results.best_params_}')
  print("\n")
  mean_score = results.cv_results_['mean_test_score']
  std_score = results.cv_results_['std_test_score']
  params = results.cv_results_['params']
  for mean, std, params in zip(mean_score, std_score, params):
    print(f'{round(mean,4)} + or - {round(std,4)} for the {params}')
display(cv)